In [1]:
#Part 9 - Intro to Encrypted Programs

import random 
import torch
import syft as sy


Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\ProgramData\Anaconda3\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


In [2]:
#Step 1: Encryption Using Secure Multi-Party Computation

Q = 1234567891011
x = 25

In [3]:
def encrypt(x):
    share_a = random.randint(-Q,Q)
    share_b = random.randint(-Q,Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)

In [4]:
encrypt(x)

(-4315078858, 723150005859, 515732964035)

In [5]:
def decrypt(*shares):
    return sum(shares) % Q

In [6]:
a,b,c = encrypt(25)


In [7]:
decrypt(a, b, c)

25

In [8]:
decrypt(a, b)

654341468273

In [9]:
#Step 2: Basic Arithmetic Using SMPC¶

x = encrypt(25)
y = encrypt(5)

In [10]:
def add(x, y):
    z = list()
    # the first worker adds their shares together
    z.append((x[0] + y[0]) % Q)
    
    # the second worker adds their shares together
    z.append((x[1] + y[1]) % Q)
    
    # the third worker adds their shares together
    z.append((x[2] + y[2]) % Q)
    
    return z

In [11]:
decrypt(*add(x,y))

30

In [12]:
#Step 3: SMPC Using PySyft

hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
bill = sy.VirtualWorker(hook, id="bill")

In [13]:
x = torch.tensor([25])

In [14]:
x

tensor([25])

In [15]:
encrypted_x = x.share(bob, alice, bill)
encrypted_x.get()

tensor([25])

In [17]:
list(bob._objects.values())

[]

In [19]:
x = torch.tensor([25]).share(bob, alice, bill)

In [20]:
# Bob's share
bobs_share = list(bob._objects.values())[0]
bobs_share

tensor([1007484311837408254])

In [21]:
# Alice's share
alices_share = list(alice._objects.values())[0]
alices_share

tensor([711035094839924888])

In [22]:
# Bill's share
bills_share = list(bill._objects.values())[0]
bills_share

tensor([2893166611750054787])

In [23]:
(bobs_share + alices_share + bills_share)

tensor([4611686018427387929])

In [24]:
x = torch.tensor([25]).share(bob,alice)
y = torch.tensor([5]).share(bob,alice)

In [25]:
z = x + y
z.get()

tensor([30])

In [26]:
z = x - y
z.get()

tensor([20])

In [27]:
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [28]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [29]:
# multiplication

z = x * y
z.get()

tensor([125])

In [30]:
x = torch.tensor([[1, 2],[3,4]]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([[2, 0],[0,2]]).share(bob,alice, crypto_provider=crypto_provider)

In [31]:
# matrix multiplication

z = x.mm(y)
z.get()

tensor([[2, 4],
        [6, 8]])

In [32]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [33]:
z = x > y
z.get()

tensor([1])

In [34]:
z = x <= y
z.get()

tensor([0])

In [35]:
z = x == y
z.get()

tensor([0])

In [36]:
z = x == y + 20
z.get()

tensor([1])

In [37]:
x = torch.tensor([2, 3, 4, 1]).share(bob,alice, crypto_provider=crypto_provider)
x.max().get()

tensor([4])

In [38]:
x = torch.tensor([[2, 3], [4, 1]]).share(bob,alice, crypto_provider=crypto_provider)
max_values, max_ids = x.max(dim=0)
max_values.get()

tensor([4, 3])